In [1]:
!pip install pycaret

In [1]:
from pycaret.datasets import get_data
import pandas as pd

In [2]:
df = get_data("titanic")
df = df.drop(["Name", "PassengerId", "Ticket", "Cabin"], axis = 1)
df["Sex"] = pd.factorize(df["Sex"])[0]
dummy = pd.get_dummies(df['Embarked'], prefix='Cabin')
df = pd.concat([df.drop("Embarked", axis=1), dummy], axis = 1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin_C,Cabin_Q,Cabin_S
0,0,3,0,22.0,1,0,7.2500,0,0,1
1,1,1,1,38.0,1,0,71.2833,1,0,0
2,1,3,1,26.0,0,0,7.9250,0,0,1
3,1,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,0,35.0,0,0,8.0500,0,0,1


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [5]:
from sklearn.model_selection import train_test_split
y = df["Survived"]
X = df.drop("Survived", axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

fill_age = X_train["Age"].mean()
X_train["Age"] = X_train["Age"].fillna(fill_age)
X_test["Age"] = X_test["Age"].fillna(fill_age)

<ipython-input-5-afadb4f713a4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["Age"] = X_train["Age"].fillna(fill_age)
<ipython-input-5-afadb4f713a4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["Age"] = X_test["Age"].fillna(fill_age)


In [10]:
from sklearn.linear_model import LogisticRegression
from typing import Any

def train_model(model: Any, X_train, X_test, y_train, y_test):
    clf = model.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return {"model": model.__class__.__name__, "params": model.get_params(), "accuracy": acc}

train_model(LogisticRegression(), X_train, X_test, y_train, y_test)

/opt/miniconda3/envs/prefect/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'model': 'LogisticRegression',
 'params': {'C': 1.0,
  'class_weight': None,
  'dual': False,
  'fit_intercept': True,
  'intercept_scaling': 1,
  'l1_ratio': None,
  'max_iter': 100,
  'multi_class': 'auto',
  'n_jobs': None,
  'penalty': 'l2',
  'random_state': None,
  'solver': 'lbfgs',
  'tol': 0.0001,
  'verbose': 0,
  'warm_start': False},
 'accuracy': 0.8100558659217877}

In [11]:
train_model(KNeighborsClassifier(), X_train, X_test, y_train, y_test)

{'model': 'KNeighborsClassifier',
 'params': {'algorithm': 'auto',
  'leaf_size': 30,
  'metric': 'minkowski',
  'metric_params': None,
  'n_jobs': None,
  'n_neighbors': 5,
  'p': 2,
  'weights': 'uniform'},
 'accuracy': 0.6871508379888268}

In [12]:
from prefect import Flow, task, unmapped
from typing import Any
import prefect

@task(nout=4)
def create_data():
    df = get_data("titanic")
    df = df.drop(["Name", "PassengerId", "Ticket", "Cabin"], axis = 1)
    df["Sex"] = pd.factorize(df["Sex"])[0]
    dummy = pd.get_dummies(df['Embarked'], prefix='Cabin')
    df = pd.concat([df.drop("Embarked", axis=1), dummy], axis = 1)
    y = df["Survived"]
    X = df.drop("Survived", axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    fill_age = X_train["Age"].mean()
    X_train["Age"] = X_train["Age"].fillna(fill_age)
    X_test["Age"] = X_test["Age"].fillna(fill_age)
    return X_train, X_test, y_train, y_test

@task
def get_models():
    return [LogisticRegression(random_state=42),
            KNeighborsClassifier(), DecisionTreeClassifier(), SVC(), 
            RandomForestClassifier(n_estimators=200, max_depth=4, random_state=42),
            RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)]

@task
def train_model(model: Any, X_train, X_test, y_train, y_test):
    clf = model.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return {"model": model.__class__.__name__, "params": model.get_params(), "accuracy": acc}

@task
def get_results(results):
    res = pd.DataFrame(results)
    prefect.context.logger.info(res)
    return res

with Flow("distributed") as flow:
    X_train, X_test, y_train, y_test = create_data()
    models = get_models()
    training_runs = train_model.map(models, unmapped(X_train), unmapped(X_test), unmapped(y_train), unmapped(y_test))
    get_results(training_runs)

In [13]:
from prefect.executors import LocalDaskExecutor
flow.executor = LocalDaskExecutor()

In [14]:
flow.run()

[2022-01-12 23:37:56-0500] INFO - prefect.FlowRunner | Beginning Flow run for 'distributed'
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'create_data': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'get_models': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'get_models': Finished task run for task with final state: 'Success'


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'create_data': Finished task run for task with final state: 'Success'
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'create_data[0]': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'create_data[1]': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'create_data[2]': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'create_data[3]': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'create_data[0]': Finished task run for task with final state: 'Success'
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'create_data[1]': Finished task run for task with final state: 'Success'
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'create_data[2]': Finished task run for task with final state: 'Success'
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'create_data[3]

<ipython-input-12-d77d46b0d383>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["Age"] = X_train["Age"].fillna(fill_age)
<ipython-input-12-d77d46b0d383>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["Age"] = X_test["Age"].fillna(fill_age)


[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'train_model[1]': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'train_model[2]': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'train_model[3]': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'train_model[4]': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'train_model[5]': Starting task run...
[2022-01-12 23:37:56-0500] INFO - prefect.TaskRunner | Task 'train_model[2]': Finished task run for task with final state: 'Success'
[2022-01-12 23:37:57-0500] INFO - prefect.TaskRunner | Task 'train_model[3]': Finished task run for task with final state: 'Success'
[2022-01-12 23:37:57-0500] INFO - prefect.TaskRunner | Task 'train_model[1]': Finished task run for task with final state: 'Success'
[2022-01-12 23:37:57-0500] INFO - prefect.TaskRunner | Task 'train_model[0]': Finished task run for task w

/opt/miniconda3/envs/prefect/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[2022-01-12 23:37:57-0500] INFO - prefect.TaskRunner | Task 'get_results': Starting task run...
[2022-01-12 23:37:57-0500] INFO - prefect.get_results |                     model                                             params  \
0      LogisticRegression  {'C': 1.0, 'class_weight': None, 'dual': False...   
1    KNeighborsClassifier  {'algorithm': 'auto', 'leaf_size': 30, 'metric...   
2  DecisionTreeClassifier  {'ccp_alpha': 0.0, 'class_weight': None, 'crit...   
3                     SVC  {'C': 1.0, 'break_ties': False, 'cache_size': ...   
4  RandomForestClassifier  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...   
5  RandomForestClassifier  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...   

   accuracy  
0  0.810056  
1  0.687151  
2  0.798883  
3  0.659218  
4  0.821229  
5  0.804469  
[2022-01-12 23:37:57-0500] INFO - prefect.TaskRunner | Task 'get_results': Finished task run for task with final state: 'Success'
[2022-01-12 23:37:57-0500] INFO - prefect.FlowRunner | Flow

<Success: "All reference tasks succeeded.">

In [16]:
!pip install tune

  Using cached tune-0.0.9-py3-none-any.whl (86 kB)


In [87]:
from tune import Space, Grid, RandInt

models = []

@task
def get_models():
    space1 = Space(model=LogisticRegression, solver="lbfgs", C=Grid(10,20), penalty=Grid("l2","none"))
    space2 = Space(model=RandomForestClassifier, max_samples=Rand(0.8,1), max_depth=RandInt(3,4)).sample(4)
    space = [x.simple_value for x in list(space1+space2)]

    for model_params in space:
        model = model_params.pop("model")
        models.append(model(**model_params))

[{'model': sklearn.linear_model._logistic.LogisticRegression,
  'solver': 'lbfgs',
  'C': 10,
  'penalty': 'l2'},
 {'model': sklearn.linear_model._logistic.LogisticRegression,
  'solver': 'lbfgs',
  'C': 10,
  'penalty': 'none'},
 {'model': sklearn.linear_model._logistic.LogisticRegression,
  'solver': 'lbfgs',
  'C': 20,
  'penalty': 'l2'},
 {'model': sklearn.linear_model._logistic.LogisticRegression,
  'solver': 'lbfgs',
  'C': 20,
  'penalty': 'none'},
 {'model': sklearn.ensemble._forest.RandomForestClassifier,
  'max_samples': 0.8342145835631154,
  'max_depth': 4},
 {'model': sklearn.ensemble._forest.RandomForestClassifier,
  'max_samples': 0.9162081028936726,
  'max_depth': 4},
 {'model': sklearn.ensemble._forest.RandomForestClassifier,
  'max_samples': 0.8335754364166126,
  'max_depth': 4},
 {'model': sklearn.ensemble._forest.RandomForestClassifier,
  'max_samples': 0.9282976548248757,
  'max_depth': 3}]

In [85]:
for model_params in space:
    model = model_params.pop("model")
    model(**model_params)